Set the environment and import the data:

In [1]:
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import DocBin
import pandas as pd
import key_dict as k

In [2]:
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
matcher = Matcher(nlp.vocab)
phraseMatcher = PhraseMatcher(nlp.vocab, attr="LEMMA")

In [ ]:
df = pd.read_csv("Tweets\sampled_tweets.xlsx")
data = df[df["retweeted"] == False] # to exclude retweets

Choose units to collect (sentence only or whole documents):

In [ ]:
 # to collect sentence only
matched_sents = [] 

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    matched_sents.append(sent.text)

In [4]:
 # to collect entire doc/cell
matched_docs = [] 

def collect_docs(matcher, doc, i, matches):
    matched_docs.append(doc.text)

Import selected patterns from keywords_and_patterns.py:

In [5]:
# Specify the patterns to add to the matcher
matcher.add("VERBS", k.belief_patterns["verbs"], on_match=collect_docs)
matcher.add("NOUNS", k.belief_patterns["nouns"], on_match=collect_docs)
matcher.add("ADVERBS", k.belief_patterns["adverbs"], on_match=collect_docs)
matcher.add("ADJECTIVES", k.belief_patterns["adjectives"], on_match=collect_docs)
matcher.add("SURE", k.belief_patterns["sure"], on_match=collect_docs)
phraseMatcher.add("PHRASES", k.belief_patterns["phrases"], on_match=collect_docs)

Compute the `doc` object on the dataset. This should only be done when the dataset changes as it can take time. 

In [ ]:
doc = data["text"].apply(lambda x: nlp(str(x)))
doc.__len__()

Once the `doc` object has been created, save it to disk:

In [ ]:
docbin = DocBin(docs=doc)
docbin.__len__()
docbin.to_disk(path="docbin.spacy")

Load saved `doc`. Make sure the vocabulary used to create the `doc` matches the loaded vocabulary (see `nlp = spacy.load()`).  Then, convert it to a list to make it iterable.

In [ ]:
docbin_loaded = DocBin().from_disk(path="docbin.spacy")
docbin_loaded

In [ ]:
docs_loaded = list(docbin_loaded.get_docs(nlp.vocab))
docs_loaded.__len__()

Load the matchers on the single docs.

In [ ]:
for x in docs_loaded:
    matcher(x)
    phraseMatcher(x)
len(matched_docs)

Delete duplicates (docs that satisfy more than one matcher):

In [ ]:
output = [i for n, i in enumerate(matched_docs) if i not in matched_docs[:n]]
len(output)

Choose what to output:

In [20]:
# to save texts only
pd.DataFrame(output).to_excel("output.xlsx", index=False, header=False)

In [13]:
# to save entire rows (this includes Twitter metadata)
boolean_series = df.text.isin(output)
filtered_df = df[boolean_series]
pd.DataFrame(filtered_df).to_excel("output.xlsx", index=False, header=False)